# Implement Multi Agent Framework

In [1]:
!pip install python-dotenv --quiet
!pip install tenacity --quiet
!pip install openai --quiet

In [2]:
from openai import OpenAI
from typing import Optional
import os
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
from typing import List, Optional, Dict
import logging
import json

In [3]:
from utils import function_to_schema

### LLM Preparation 

In [101]:
# Load environment variables from .env file
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

### Agent & Multi-Agent 

In [213]:

class BaseAgent:
    """
    An example of a base agent class, including an OpenAI client.
    """
    def __init__(self,client:OpenAI):
        self.client = client
        # can add more attributes and methods here
        

class Agent:
    """
    A class representing an agent that can handle chat messages and handoff to other agents.
    """
    def __init__(
            self,
            base_agent:BaseAgent,
            name:str,
            instructions: Optional[str] = None,
            handoff_description: Optional[str] = None,
            ):
        self.DEFAULT_MODEL = "gpt-4o-mini"
        self.name = name.replace(" ","_")
        self.agent = base_agent
        self.instructions = instructions
        self.handoff_description = handoff_description
        self.handoff_agent_schemas: List[Dict] = []
        self.tools_schema: List[Dict] = []
        self.handoff_agents: Dict[str, "Agent"] = {}
        self.tools_map: Dict[str, "function"] = {}

    def __str__(self):
        """
        Returns a string representation of the Agent instance.
        """
        return f"Agent(name={self.name}, instructions={self.instructions},handoff_agents={list(self.handoff_agents.keys())})"


    def add_handoffs(self, agents: List["Agent"]) -> None:
        """ 
        Adds handoff agents to the current agent.
        """
        self.handoff_agent_schemas.clear()
        for agent in agents:
            agent_schema = {
                            "type": "function",
                            "function": {
                                "name": agent.name,
                                "description": agent.instructions or "" + " \n" + agent.handoff_description or "",
                                'parameters': {
                                    "type": "object",
                                    "properties": {
                                        "message": {
                                            "type": "string",
                                            "description": "The message to send to the agent."
                                            }
                                    },
                                    "required": ["message"]
                                }
                            }
                            }
            self.handoff_agent_schemas.append(agent_schema)
            self.handoff_agents[agent.name] = agent

    def clear_handoffs(self) -> None:
        """ 
        Clears handoff agents from the current agent.
        """
        self.handoff_agent_schemas.clear()
        self.handoff_agents = {}

    def add_tools(self, tools: list) -> None:
        """ 
        Adds tools to the current agent.
        """
        self.tools_schema.clear()
        for tool in tools:
            tool_schema = function_to_schema(tool)
            self.tools_schema.append(tool_schema)
            self.tools_map[tool.__name__] = tool

    def clear_tools(self) -> None:
        """
        Clears tools from the current agent.
        """
        self.tools_schema.clear()

    @retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
    def chat(self,messages:List[Dict],model: Optional[str] = None,disable_tools:bool = False,disable_handoffs:bool = False):
        """ 
        Handles chat messages and returns responses, with optional handoff to other agents.
        """

        if self.instructions:
            messages = [{"role": "system", "content": self.instructions}] + messages
        
        model = model or self.DEFAULT_MODEL

        try:

            
            tools = None if disable_tools else self.tools_schema or None
            handoffs = None if disable_handoffs else self.handoff_agent_schemas or None

            response = self.agent.client.chat.completions.create(
                            model=model,
                            messages=messages,
                            tools=tools + handoffs if tools and handoffs else tools or handoffs,
                            tool_choice=None,
                        )
            message = response.choices[0].message
            if not message.tool_calls:
                result = [{"role": "assistant", "content": message.content, "agent_name": self.name}]
                return result
            
            result = []
            handoff_agents_num = sum([1 for tool_call in message.tool_calls if tool_call.function.name in self.handoff_agents])
            for tool_call in message.tool_calls:
                if tool_call.function.name in self.handoff_agents:
                    handoff_agent = self.handoff_agents[tool_call.function.name]
                    if handoff_agents_num > 1:
                        message = json.loads(tool_call.function.arguments).get("message")
                    else:
                        message = messages[-1].get("content")
                    handoff_message = {
                        "role": "handoff",
                        "handoff": f"{self.name} -> {handoff_agent.name}",
                        "agent_name": handoff_agent.name,
                        "agent": handoff_agent,
                        "message": message,
                    }
                    result.append(handoff_message)
                elif tool_call.function.name in self.tools_map:
                    tool = self.tools_map[tool_call.function.name]
                    tool_args = json.loads(tool_call.function.arguments)
                    tool_result = tool(**tool_args)
                    tool_message = {
                        "role": "tool",
                        "content": tool_result,
                    }
                    result.append(tool_message)
                    logging.info(f"Tool call: {tool_call.function.name} with args: {tool_args} returned: {tool_result}")
                else:
                    logging.error(f"Unknown tool call: {tool_call.function.name}")
            return result
        except Exception as e:
            logging.error("Unable to generate ChatCompletion response")
            logging.error(f"Exception: {e}")
            return None


class MultiAgent:
    def __init__(self,start_agent:Optional[Agent] = None):
        """
        Initialize the MultiAgent with an optional starting agent.
        """
        self.current_agent = start_agent

    def add_handoff_relations(self,from_agent:Agent,to_agents:List[Agent])->None:
        """
        Add handoff relations from one agent to multiple agents.
        """
        from_agent.add_handoffs(to_agents)

    def chat(self,
             messages:List[Dict],
             model: Optional[str] = None,
             agent:Optional[Agent] = None,
             max_handoof_depth:int = 2,
             disable_tools:bool = False,
             disable_handoffs:bool = False
             )->List[Dict]:
        """
        Chat with the current agent or a specified agent using the provided messages and model.
        """
        if agent:
            self.current_agent = agent

        if not self.current_agent:
            logging.error("No agent is set for conversation. You can set it in init or pass as argument.")
            return None
        
        try:
            # currently only one handoff is supported
            res = self.current_agent.chat(messages, model, disable_tools=disable_tools, disable_handoffs=disable_handoffs)

            if not res:
                return res

            if len(res) == 1:
                # Check if initial response has content
                if res[0].get("content"):
                    return res
                # Handle handoff
                while res[-1].get("handoff") and max_handoof_depth > 0:
                    max_handoof_depth -= 1
                    self.current_agent = res[-1]["agent"]
                    disable_handoffs = max_handoof_depth == 0
                    res.extend(self.current_agent.chat(messages, model, disable_handoffs=disable_handoffs))

                return res
            else:
                for r in res:
                    if r.get("handoff"):
                        temp_agent = r["agent"]
                        temp_messages = messages.copy()
                        temp_messages[-1]["content"] = r["message"]
                        res.extend(temp_agent.chat(temp_messages, model, disable_handoffs=True))
                return res
        except Exception as e:
            logging.error("Error during chat with agent.")
            logging.error(f"Exception: {e}")
            return None
        
    def handoff(self,messages:List[Dict],model: Optional[str] = None,agent:Agent = None)-> Agent:
        """
        Perform a handoff to the specified agent using the provided messages and model.
        """
        current_agent = agent

        if not current_agent:
            logging.error("No agent is set for conversation. You must specify an agent.")
            return None
        
        try:
            res = current_agent.chat(messages, model)
            handoff_agents = {}
            if res:
                for r in res:
                    if r.get("handoff"):
                        handoff_agents[r["agent_name"]] = r["agent"]
            if handoff_agents:
                return handoff_agents
            return None
        except Exception as e:
            logging.error("Error during handoff with agent.")
            logging.error(f"Exception: {e}")
            return None

### Init Several Agents

In [231]:
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
           handoff_description = "Call this agent if a user is asking about a topic that is not handled by the activated agent.")

science_agent = Agent(base_agent = BaseAgent(client),
           name = "science agent",
           instructions = "Agent used for science questions named Albert.",
           handoff_description = "Call this agent if a user is asking about a science topic.")

music_agent = Agent(base_agent = BaseAgent(client),
           name = "music agent",
           instructions = "Agent used for music questions named Mozart.",
           handoff_description = "Call this agent if a user is asking about a music topic.")

daily_agent = Agent(base_agent = BaseAgent(client),
           name = "daily agent",
           instructions = "Agent used for daily questions named Ada.",
           handoff_description = "Call this agent if a user is asking about a daily topic.")

###  Build a Multi-Agent environment & add handoff relationships 

In [232]:
# Build a Multi-Agent environment & add handoff relationships
ma = MultiAgent(start_agent=general_agent)
ma.add_handoff_relations(from_agent=general_agent,to_agents=[science_agent,music_agent,daily_agent])
ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent])
ma.add_handoff_relations(from_agent=daily_agent,to_agents=[general_agent])
# ma.add_handoff_relations(from_agent=science_agent,to_agents=[general_agent,music_agent,daily_agent])
# ma.add_handoff_relations(from_agent=music_agent,to_agents=[general_agent,science_agent,daily_agent])
# ma.add_handoff_relations(from_agent=daily_agent,to_agents=[general_agent,science_agent,music_agent])



### Multi-Agent handoff example

use handoff to get the agent who is suitable for the task

Then call the agent's pre-designed capabilities, such as streaming, tools, memory, reasoning, etc.

In [233]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue"}],agent=general_agent)

{'science_agent': <__main__.Agent at 0x114341390>}

In [234]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

{'science_agent': <__main__.Agent at 0x114341390>,
 'music_agent': <__main__.Agent at 0x11437d350>}

### Multi-Agent dialogue example

In [235]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue"}])

[{'role': 'handoff',
  'handoff': 'general_agent -> science_agent',
  'agent_name': 'science_agent',
  'agent': <__main__.Agent at 0x114341390>,
  'message': 'why the sky is blue'},
 {'role': 'assistant',
  'content': "The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it interacts with molecules and small particles in the air. Sunlight, or white light, is made up of different colors, each with its own wavelength. Blue light has a shorter wavelength and is scattered more than other colors when it strikes air molecules. \n\nAs a result, when we look up at the sky, we see more of the blue light scattered in all directions, making the sky appear blue during the day. When the sun is low on the horizon during sunrise or sunset, the light has to pass through more of the atmosphere, scattering the blue and green light and allowing the reds and oranges to become more prominent, creating the beautiful colors often seen at those time

In [236]:
ma.chat(messages=[{"role": "user", "content": "who are you"}])

[{'role': 'assistant',
  'content': 'I am Albert, your assistant for science-related questions. How can I help you today?',
  'agent_name': 'science_agent'}]

In [237]:
ma.chat(messages=[{"role": "user", "content": "recommend me some music"}])

[{'role': 'handoff',
  'handoff': 'science_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x114343c10>,
  'message': 'recommend me some music'},
 {'role': 'handoff',
  'handoff': 'general_agent -> music_agent',
  'agent_name': 'music_agent',
  'agent': <__main__.Agent at 0x11437d350>,
  'message': 'recommend me some music'},
 {'role': 'assistant',
  'content': 'Sure! Here are some recommendations across various genres:\n\n**Classical:**\n- **Ludwig van Beethoven** – Symphony No. 9 in D minor, Op. 125 (“Choral”)\n- **Johann Sebastian Bach** – Brandenburg Concerto No. 3\n- **Wolfgang Amadeus Mozart** – Symphony No. 40 in G minor, K. 550\n\n**Jazz:**\n- **Miles Davis** – Kind of Blue\n- **John Coltrane** – A Love Supreme\n- **Ella Fitzgerald & Louis Armstrong** – Ella and Louis\n\n**Rock:**\n- **The Beatles** – Abbey Road\n- **Led Zeppelin** – IV\n- **Fleetwood Mac** – Rumours\n\n**Pop:**\n- **Dua Lipa** – Future Nostalgia\n- **Taylor Swift** – 198

In [238]:
ma.chat(messages=[{"role": "user", "content": "dinner tonight"}])

[{'role': 'handoff',
  'handoff': 'music_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x114343c10>,
  'message': 'dinner tonight'},
 {'role': 'handoff',
  'handoff': 'general_agent -> daily_agent',
  'agent_name': 'daily_agent',
  'agent': <__main__.Agent at 0x114565b50>,
  'message': 'dinner tonight'},
 {'role': 'assistant',
  'content': 'What are you in the mood for? Here are a few ideas:\n\n1. **Pasta**: A classic spaghetti carbonara or a creamy pesto dish.\n2. **Stir-fry**: Quick and easy with your choice of protein and vegetables.\n3. **Tacos**: Load them with chicken, beef, or beans, and top with your favorite toppings.\n4. **Salad**: A hearty salad with grilled chicken or roasted veggies can be refreshing.\n5. **Soup**: Cozy up with a warm bowl of soup, like a hearty chili or creamy tomato.\n\nLet me know if you want a specific recipe or need more suggestions!',
  'agent_name': 'daily_agent'}]

In [246]:
ma.chat(messages=[{"role": "user", "content": "who are you"}],agent=science_agent)

[{'role': 'assistant',
  'content': 'I am Albert, your assistant for science-related questions. How can I help you today?',
  'agent_name': 'science_agent'}]

In [241]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=general_agent)

[{'role': 'handoff',
  'handoff': 'general_agent -> science_agent',
  'agent_name': 'science_agent',
  'agent': <__main__.Agent at 0x114341390>,
  'message': 'Why is the sky blue?'},
 {'role': 'handoff',
  'handoff': 'general_agent -> music_agent',
  'agent_name': 'music_agent',
  'agent': <__main__.Agent at 0x11437d350>,
  'message': 'Can you recommend me some music?'},
 {'role': 'assistant',
  'content': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. Sunlight, or white light, is made up of many colors, each with different wavelengths. When sunlight enters Earth's atmosphere, it interacts with gas molecules and small particles.\n\nShorter wavelengths of light (blue and violet) are scattered more than longer wavelengths (red and yellow). Although violet light is scattered even more than blue, our eyes are more sensitive to blue light and some of the violet is absorbed by the ozone layer. As a result, we perceive the sky as blue during the day.\n\nDuring

### Multi-Agent tool use example

In [247]:
def get_weather(city:str)->str:
    """ 
    Get the weather for a specified city.
    """
    return f"The weather in {city} is sunny."


def get_news()->str:
    """ 
    Get the latest news.
    """
    return "The latest news is that the sun is shining."

In [248]:
daily_agent.add_tools([get_weather,get_news])

In [249]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou?"}])

[{'role': 'handoff',
  'handoff': 'science_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x114343c10>,
  'message': 'what is the weather in Hangzhou?'},
 {'role': 'handoff',
  'handoff': 'general_agent -> daily_agent',
  'agent_name': 'daily_agent',
  'agent': <__main__.Agent at 0x114565b50>,
  'message': 'what is the weather in Hangzhou?'},
 {'role': 'tool', 'content': 'The weather in Hangzhou is sunny.'}]

In [228]:
ma.chat(messages=[{"role": "user", "content": "what happened today?"}])

[{'role': 'tool', 'content': 'The latest news is that the sun is shining.'}]

In [250]:
ma.chat(messages=[{"role": "user", "content": "what is the weather in Hangzhou and what happened today?"}])

[{'role': 'tool', 'content': 'The weather in Hangzhou is sunny.'},
 {'role': 'tool', 'content': 'The latest news is that the sun is shining.'}]